In [1]:
# импортируем библиотеки numpy и pandas
import warnings

warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from pandas import Series, DataFrame
from collections import Counter, defaultdict
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import KBinsDiscretizer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import norm
from scipy import stats
from sklearn.preprocessing import PowerTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from numpy.random import RandomState
#Загружаем данные
data = pd.read_excel('credit_train.xlsx')



In [2]:
#Устанавливаем подходящие параметры для отображения стобцов
pd.set_option('display.max_columns', 60)

In [3]:
#Заменяем пропущенные значения в age, score_shk, monthly_income и credit_sum 
data['age'] = np.where(data['age'].isnull(), data['age'].median(), data['age'])
data['score_shk'] = np.where(data['score_shk'].isnull(), data['score_shk'].median(), data['score_shk'])
data['credit_sum'] = np.where(data['credit_sum'].isnull(), data['credit_sum'].median(), data['credit_sum'])
data['monthly_income'] = np.where(data['monthly_income'].isnull(), data['monthly_income'].median(), data['monthly_income'])

In [4]:
#Удаляем столбец client_id так как он представляет собой просто номер id клиента
data.drop('client_id', axis=1, inplace=True)

In [5]:
#Быстрый взгляд на данные
data.head()

,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,M,34.0,NaN,UMN,59998.00,10,2019-06-01 00:00:00,0.461599,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,F,34.0,MAR,UMN,10889.00,6,2019-01-01 00:00:00,0.461599,NaN,МОСКВА,35000.0,2.0,0.0,0
2,M,32.0,MAR,SPC,10728.00,12,2019-01-01 00:00:00,0.461599,NaN,ОБЛ САРАТОВСКАЯ,35000.0,5.0,0.0,0
3,F,27.0,NaN,SPC,12009.09,12,2019-01-01 00:00:00,0.461599,NaN,ОБЛ ВОЛГОГРАДСКАЯ,35000.0,2.0,0.0,0
4,M,45.0,NaN,SPC,21229.00,10,2019-01-01 00:00:00,0.421385,SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,35000.0,1.0,0.0,0


In [6]:
#Работа с метками регионов

In [7]:
#Группируем синонимичные значения регионов
data['living_region'] = np.where((data['living_region'] == 'КРАЙ КРАСНОДАРСКИЙ')
                                 |(data['living_region'] == 'КРАСНОДАРСКИЙ'), 'КРАСНОДАРСКИЙ КРАЙ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'МОСКВА Г')
                                 |(data['living_region'] == 'Г МОСКВА')
                                 |(data['living_region'] == 'Г.МОСКВА')
                                 |(data['living_region'] == 'Г. МОСКВА'), 'МОСКВА', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'ОБЛ.МОСКОВСКАЯ')
                                 |(data['living_region'] == 'МОСКОВСКАЯ ОБЛ')
                                 |(data['living_region'] == 'МОСКОВСКАЯ')
                                 |(data['living_region'] == 'МОСКВОСКАЯ ОБЛ')
                                 |(data['living_region'] == 'МОСКОВСКАЯ ОБЛАСТЬ'), 'ОБЛ МОСКОВСКАЯ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'САНКТ-ПЕТЕРБУРГ Г')
                                 |(data['living_region'] == 'Г. САНКТ-ПЕТЕРБУРГ'), 'САНКТ-ПЕТЕРБУРГ', data['living_region'])


data['living_region'] = np.where((data['living_region'] == 'РЕСП ТАТАРСТАН')
                                 |(data['living_region'] == 'РЕСПУБЛИКАТАТАРСТАН')
                                 |(data['living_region'] == 'РЕСПУБЛИКА ТАТАРСТАН'), 'ТАТАРСТАН РЕСП', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'РЕСП.БАШКОРТОСТАН')
                                 |(data['living_region'] == 'РЕСП. БАШКОРТОСТАН')
                                 |(data['living_region'] == 'БАШКОРТОСТАН')
                                 |(data['living_region'] == 'БАШКОРТОСТАН РЕСП'), 'РЕСП БАШКОРТОСТАН', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'ИРКУТСКАЯ ОБЛ')
                                 |(data['living_region'] == 'ИРКУТСКАЯ ОБЛАСТЬ'), 'ОБЛ ИРКУТСКАЯ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'ОБЛ СВЕРДЛОВСКАЯ')
                                 |(data['living_region'] == 'ОБЛ. СВЕРДЛОВСКАЯ')
                                 |(data['living_region'] == 'СВЕРДЛОВСКАЯ')
                                 |(data['living_region'] == 'СВЕРДЛОВСКАЯ ОБЛАСТЬ'), 'СВЕРДЛОВСКАЯ ОБЛ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'ОБЛ.НИЖЕГОРОДСКАЯ')
                                 |(data['living_region'] == 'НИЖЕГОРОДСКАЯ ОБЛ'), 'ОБЛ НИЖЕГОРОДСКАЯ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'САХА /ЯКУТИЯ/')
                                 |(data['living_region'] == 'САХА (ЯКУТИЯ)'), 'САХА', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'ЧУВАШСКАЯ  - ЧУВАШИЯ')
                                 |(data['living_region'] == 'ЧУВАШСКАЯ - ЧУВАШИЯ')
                                 |(data['living_region'] == 'ЧУВАШИЯ ЧУВАШСКАЯ  -')
                                 |(data['living_region'] == 'ЧУВАШСКАЯ'), 'ЧУВАШИЯ', data['living_region'])

data['living_region'] = np.where((data['living_region'].isnull())|(data['living_region'] == 'РОССИЯ')|(data['living_region'] == 'ПРИВОЛЖСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ'), 'Центральный', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'САХА /ЯКУТИЯ/')|
                                 (data['living_region'] == 'САХА (ЯКУТИЯ)'), 'САХА', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА')|
                                 (data['living_region'] == 'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - Ю')|
                                 (data['living_region'] == 'ХАНТЫ-МАНСИЙСКИЙ'), 'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'ЕВРЕЙСКАЯБЛ')|
                                 (data['living_region'] == 'ЕВРЕЙСКАЯ АВТОНОМНАЯ')|
                                 (data['living_region'] == 'ЕВРЕЙСКАЯ АВТОНОМНАЯ'), 'ЕВРЕЙСКАЯ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'АЛТАЙСКИЙ'), 'АЛТАЙ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'БРЯНСКАЯ'), 'БРЯНСКИЙ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'ГОДИНЦОВО МОСКОВСКАЯ'), 'МОСКОВСКАЯ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'ГУСЬ-ХРУСТАЛЬНЫЙ Р-Н'), 'ВЛАДИМИРСКАЯ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'ГОРЬКОВСКАЯ'), 'НИЖЕГОРОДСКАЯ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'КАМЧАТСКИЙ'), 'КАМЧАТСКАЯ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'МЫТИЩИНСКИЙ Р-Н'), 'МОСКОВСКАЯ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'ОБЛРОСТОВСКАЯ'), 'РОСТОВСКАЯ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'ОБЛСАРАТОВСКАЯ'), 'САРАТОВСКАЯ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'СЕВ ОСЕТИЯ - АЛАНИЯ'), 'СЕВЕРНАЯ ОСЕТИЯ - АЛАНИЯ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'НЕНЕЦКИЙ'), 'ЯМАЛО-НЕНЕЦКИЙ', data['living_region'])

data['living_region'] = np.where((data['living_region'] == 'САХА'), 'САХАЛИНСКАЯ', data['living_region'])

In [8]:
#Удаляем ненужные метки из названий регионов
data['living_region'] = data['living_region'].str.replace('КРАЙ', '')
data['living_region'] = data['living_region'].str.replace('РЕСПУБЛИКА', '')
data['living_region'] = data['living_region'].str.replace('РЕСП.', '')
data['living_region'] = data['living_region'].str.replace('РЕСП', '')
data['living_region'] = data['living_region'].str.replace('ОБЛАСТЬ', '')
data['living_region'] = data['living_region'].str.replace(' ОБЛАСТЬ', '')
data['living_region'] = data['living_region'].str.replace('ОБЛ ', '')
data['living_region'] = data['living_region'].str.replace(' ОБЛ', '')
data['living_region'] = data['living_region'].str.replace('ОБЛ. ', '')
data['living_region'] = data['living_region'].str.replace('.', '')
data['living_region'] = data['living_region'].str.strip(' ')
data['living_region'] = data['living_region'].str.replace(' АО', '')
data['living_region'] = data['living_region'].str.replace('АО ', '')
data['living_region'] = data['living_region'].str.replace('АО', '')
data['living_region'] = data['living_region'].str.strip(' ')

In [9]:
#Исправляем мелкие неточности
data['tariff_id'] = np.where(data['tariff_id'] == '1.0', '2019-01-01 00:00:00', data['tariff_id'])

In [10]:
#Теперь давайте сгруппируем наши данные по регионам в группы по экономическим зонам РФ
#Исправим несколько ошибок в регионах
data['living_region_area'] = data['living_region']

data['living_region_area'] = np.where((data['living_region_area'] == 'ЧУВАШСКАЯ  - ЧУВАШИЯ')
                                 |(data['living_region_area'] == 'ЧУВАШСКАЯ - ЧУВАШИЯ')
                                 |(data['living_region_area'] == 'ЧУВАШИЯ ЧУВАШСКАЯ  -')
                                 |(data['living_region_area'] == 'ЧУВАШСКАЯ'), 'ЧУВАШИЯ', data['living_region_area'])

data['living_region_area'] = np.where((data['living_region_area'] == 'САХА /ЯКУТИЯ/')
                                 |(data['living_region_area'] == 'САХА (ЯКУТИЯ)'), 'САХА', data['living_region_area'])


dicta = {98: 'Other'}
data['living_region'].replace(dicta, inplace=True)

In [11]:
#Создаем словарь для группировки
dicta_dicta = {'БРЯНСКАЯ': 'Центральный', 'ВЛАДИМИРСКАЯ' : 'Центральный', 'ИВАНОВСКАЯ': 'Центральный', 'КАЛУЖСКАЯ' : 'Центральный', 'КОСТРОМСКАЯ' : 'Центральный', 'МОСКОВСКАЯ' : 'Центральный', 
              'ОРЛОВСКАЯ': 'Центральный', 'РЯЗАНСКАЯ': 'Центральный', 'СМОЛЕНСКАЯ': 'Центральный', 'ТВЕРСКАЯ': 'Центральный', 'ТУЛЬСКАЯ': 'Центральный', 'ЯРОСЛАВСКАЯ': 'Центральный', 'МОСКВА': 'Центральный',
              'БЕЛГОРОДСКАЯ': 'Центрально-Чернозёмный', 'ВОРОНЕЖСКАЯ': 'Центрально-Чернозёмный', 'КУРСКАЯ': 'Центрально-Чернозёмный', 'ЛИПЕЦКАЯ': 'Центрально-Чернозёмный', 'ТАМБОВСКАЯ': 'Центрально-Чернозёмный',
              'КРАСНОДАРСКИЙ' : 'Северо-Кавказский', 'САРАТОВСКАЯ' : 'Поволжский', 'ВОЛГОГРАДСКАЯ' : 'Поволжский', 'ЧЕЛЯБИНСКАЯ': 'Уральский', 'СТАВРОПОЛЬСКИЙ': 'Поволжский', 'НИЖЕГОРОДСКАЯ': 'Волго-Вятский',
               'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - ЮГРА' : 'Западно-Сибирский', 'САНКТ-ПЕТЕРБУРГ' : 'Северо-Западный', 'БАШКОРТОСТАН': 'Поволжский', 'АРХАНГЕЛЬСКАЯ': 'Северный', 'ХАНТЫ-МАНСИЙСКИЙ': 'Западно-Сибирский', 
               'ПЕРМСКИЙ': 'Уральский', 'КАРАЧАЕВО-ЧЕРКЕССКАЯ': 'Северо-Кавказский', 'ВОЛОГОДСКАЯ' : 'Северный', 'РОСТОВСКАЯ' : 'Северо-Кавказский', 'УДМУРТСКАЯ': 'Уральский', 'ИРКУТСКАЯ': 'Восточно-Сибирский', 'ТЮМЕНСКАЯ': 'Западно-Сибирский',
               'ХАКАСИЯ': 'Восточно-Сибирский', 'ТАТАРСТАН' : 'Поволжский', 'СВЕРДЛОВСКАЯ': 'Уральский', 'ПСКОВСКАЯ': 'Северо-Западный', 'ЗАБАЙКАЛЬСКИЙ': 'Восточно-Сибирский', 'ОРЕНБУРГСКАЯ' : 'Уральский', 'АСТРАХАНСКАЯ': 'Поволжский', 'НОВОСИБИРСКАЯ' : 'Западно-Сибирский',
               'КУРГАНСКАЯ': 'Уральский', 'УЛЬЯНОВСКАЯ' : 'Поволжский', 'МУРМАНСКАЯ' : 'Северный', 'КРАСНОЯРСКИЙ' : 'Восточно-Сибирский', 'БУРЯТИЯ' : 'Восточно-Сибирский', 'САХА': 'Дальневосточный', 'АМУРСКАЯ': 'Дальневосточный', 'ХАБАРОВСКИЙ' : 'Дальневосточный', 'ЯМАЛО-НЕНЕЦКИЙ': 'Западно-Сибирский',
               'САМАРСКАЯ': 'Поволжский', 'ЛЕНИНГРАДСКАЯ': 'Северо-Западный', 'КЕМЕРОВСКАЯ' : 'Западно-Сибирский', 'ОМСКАЯ': 'Западно-Сибирский', 'ЧЕЧЕНСКАЯ' : 'Северо-Кавказский', 'АДЫГЕЯ': 'Северо-Кавказский', 'КОМИ': 'Северный', 'ПРИМОРСКИЙ' : 'Дальневосточный', 'КИРОВСКАЯ': 'Волго-Вятский', 'ДАГЕСТАН': 'Северо-Кавказский',
               'ПЕНЗЕНСКАЯ': 'Поволжский', 'КАРЕЛИЯ': 'Северный', 'ТОМСКАЯ': 'Западно-Сибирский', 'МАГАДАНСКАЯ': 'Дальневосточный', 'МАРИЙ ЭЛ': 'Волго-Вятский', 'ЕВРЕЙСКАЯБЛ': 'Дальневосточный', 'АЛТАЙСКИЙ':'Западно-Сибирский', 'КАБАРДИНО-БАЛКАРСКАЯ':'Северо-Кавказский', 'ХАНТЫ-МАНСИЙСКИЙ АВТОНОМНЫЙ ОКРУГ - Ю':'Западно-Сибирский', 'ТЫВА':'Восточно-Сибирский',
               'НОВГОРОДСКАЯ': 'Северо-Западный', 'СЕВЕРНАЯ ОСЕТИЯ - АЛАНИЯ': 'Северо-Кавказский', 'САХАЛИНСКАЯ': 'Дальневосточный', 'ЧУВАШИЯ':'Волго-Вятский', 'КАМЧАТСКИЙ': 'Дальневосточный', 'МОРДОВИЯ': 'Волго-Вятский', 'КАЛМЫКИЯ': 'Поволжский', 'АЛТАЙ': 'Западно-Сибирский', 'КАЛИНИНГРАДСКАЯ':'Центральный', 'НЕНЕЦКИЙ':'Западно-Сибирский', 'ПЕРМСКАЯ':'Уральский', 'ЧУКОТСКИЙ': 'Дальневосточный',
               'ГОРЬКОВСКАЯ': 'Волго-Вятский', 'СЕВ ОСЕТИЯ - АЛАНИЯ': 'Северо-Кавказский', 'ИНГУШЕТИЯ': 'Северо-Кавказский', 'ЧИТИНСКАЯ': 'Западно-Сибирский', 'МЫТИЩИНСКИЙ Р-Н': 'Центральный', 'ОБЛРОСТОВСКАЯ' : 'Северо-Кавказский', 'ЕВРЕЙСКАЯ АВТОНОМНАЯ': 'Дальневосточный', 'ЭВЕНКИЙСКИЙ':'Восточно-Сибирский', 'КАМЧАТСКАЯ': 'Дальневосточный', 'ГУСЬ-ХРУСТАЛЬНЫЙ Р-Н': 'Центральный', 'БРЯНСКИЙ': 'Центральный',
               'ОРЁЛ': 'Центральный', 'АЕВРЕЙСКАЯ': 'Дальневосточный', 'ОБЛСАРАТОВСКАЯ': 'Поволжский', 'ДАЛЬНИЙ ВОСТОК' : 'Дальневосточный', 'ГОДИНЦОВО МОСКОВСКАЯ': 'Центральный'
              }

data['living_region_area'].replace(dicta_dicta, inplace = True)

In [12]:
#Feature engineering

In [13]:
#Первый созданный нами признак - Payment - сумма кредита / срок кредита
data['payment'] = data['credit_sum'] / data['credit_month']

#Второй признак - share_income - доля дохода от суммы кредита
data['share_income'] = data['monthly_income'] / data['credit_sum']

#Третий признак - income_to_credit - отношение месячной зарплаты к временному периоду погашения кредита
data['income_to_credit'] = data['age'] / data['credit_month']

#Четвертый признак - retired - пенсионер или нет
data['retired'] = np.where(data['age']>=60, 'Yes', 'No')

#Пятый признак - age_retired - взрослый и с высоким уровнем дохода
data['age_retired'] = np.where((data['age'] > 35) & (data['credit_sum'] > 10000), 'Yes', 'No')

#Шестой признак - rich_and_married - человек с высоким доходом и женатый
data['rich_and_married'] = np.where((data['marital_status'] == 'MAR') & (data['monthly_income'] > 50000), 'Yes', 'No')

#Седьмой признак - rich_and_DIR - человек с высоким доходом и имеющий должность директора
data['rich_and_DIR'] = np.where((data['job_position'] == 'DIR') & (data['monthly_income'] > data['monthly_income'].median()), 'Yes', 'No')

#Восьмой признак - rich_and_INP - человек с высоким доходом и имеющий должность аналитика
data['rich_and_INP'] = np.where((data['job_position'] == 'INP') & (data['monthly_income'] > data['monthly_income'].median()), 'Yes', 'No')

#Девятый признак - credit_pressure - кредитная нагрузка
data['credit_pressure'] = data['monthly_income'] / data['payment']

In [14]:
#Устанавливаем показатели средней зарплаты
dicta_dict = {'UMN': 51000, 'SPC': 63000, 'INP': 55000, 'DIR': 60000,
'ATP': 46000, 'PNA': 71000, 'BIS': 86000, 'WOI': 76000,
'NOR': 54000, 'WRK': 77000, 'WRP': 75000, 'PNV': 67000,
'BIU': 43000, 'PNI': 69000, 'HSK': 74000, 'PNS': 44000,
'INV': 88000, 'ONB': 62000, 'OTHER': 20000}

data['average_zp'] = data['job_position'].map(dicta_dict)

In [15]:
# создаем обучающий массив признаков, обучающий массив меток,
# тестовый массив признаков, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(data.drop('open_account_flg', axis=1), 
                                                data['open_account_flg'], 
                                                test_size=.3, 
                                                stratify=data['open_account_flg'], 
                                                random_state=100)

In [16]:
#Создаем функцию для проверки признаков по AUC
def importace_auc(train, test):
    train_copy = train.copy()
    test_copy = test.copy()
    #Выбираем нужные нам признаки с числовым типом данных
    col_list = train.select_dtypes(include = ['number']).columns
    #Создаем словарь
    auc_list = []
    for i in col_list:
        #Заменяем все пропущенные значения на медианное
        train_copy[i].fillna(train_copy[i].median(), inplace = True)
        # Создаем экземляр класса PowerTransformer
        power = PowerTransformer(method = 'yeo-johnson', standardize = True).fit(train_copy[[i]])
        #Трансформируем данные тренировочной выборки
        train_copy[i] = power.transform(train_copy[[i]])
        #Заменяем все пропущенные значения на медианное
        test_copy[i].fillna(test_copy[i].median(), inplace = True)
        #Трансформируем данные тестовой выборки
        test_copy[i] = power.transform(test_copy[[i]])
        #Создаем экземпляр класса LogisticRegression
        logreg = LogisticRegression(solver = 'liblinear').fit(train_copy[[i]], y_train)
        #Подсчитываем значение AUC для каждой категории
        auc = roc_auc_score(y_test, logreg.predict_proba(test_copy[[i]])[:, 1])
        #Добавляет в наш пустой список значения auc
        auc_list.append(auc)
    #Создаем датафрейм с показателями auc
    result = pd.DataFrame({'Переменная': col_list, 'AUC': auc_list})
    #Округляем значения и сортируем по убыванию
    result = np.round(result.sort_values(by = 'AUC', ascending = False), 3)
    #Форматирование по цвету
    cm = sns.light_palette('yellow', as_cmap = True)
    return(result.style.background_gradient(cmap = cm))

In [17]:
#Смотрим на значимость категорий по показателю AUC
importace_auc(X_train, X_test)

,Переменная,AUC
7,payment,0.562
1,credit_sum,0.554
10,credit_pressure,0.543
8,share_income,0.538
3,score_shk,0.535
0,age,0.534
9,income_to_credit,0.53
2,credit_month,0.529
5,credit_count,0.523
6,overdue_credit_count,0.504


In [18]:
#Устанавливаем признак tariff_id как качественный признак
X_train['tariff_id'] = X_train['tariff_id'].astype('object')

In [19]:
# создаем собственный класс Replacer, заменяющие отрицательные
# и нулевые значения на небольшие положительные (необходимо
# перед применением логарифмического и обратного преобразований)
class Replacer(BaseEstimator, TransformerMixin):
    """
    Параметры:
        repl: значение для замены
    """
    def __init__(self, repl_value=0.1):
        self.repl_value = repl_value
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_replaced = np.where(X <= 0, self.repl_value, X)
        return X_replaced

In [20]:
#Разделяем признаки на качественные и количественные
cat_columns = X_train.select_dtypes('object').columns
num_columns = X_train.select_dtypes(include = ['int64', 'float64']).columns

In [21]:
#Трансформер для категориальных переменных
cat_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='most_frequent')),
    ('derp', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])

In [22]:
#Трансформер для количественных переменных
num_pipe = Pipeline([
    ('imp', SimpleImputer(strategy='mean')),
    ('repl', Replacer(repl_value=0.1)),
    ('yeo', PowerTransformer(method = 'box-cox', standardize = True))])

In [23]:
#Трансформер для количественных переменных
num_pipe_test = Pipeline([
    ('imp', SimpleImputer(strategy='mean')),
    ('repl', Replacer(repl_value=0.1)),
    ('yeo', PowerTransformer(method = 'box-cox', standardize = True)),
    ('bi', KBinsDiscretizer(encode='onehot-dense'))])

In [24]:
#Общий конвеер
transformers = [('cat', cat_pipe, cat_columns),
               ('num_bin', num_pipe_test, num_columns)]

transformer = ColumnTransformer(transformers=transformers)

In [25]:
transformer = ColumnTransformer(transformers=transformers)

In [26]:
#Финальный трансформер
pipe = Pipeline([('tf', transformer), 
                 ('logreg', LogisticRegression(solver='liblinear', random_state=42, C=5))])

In [27]:
#Параметры для модели
param_grid = [
    {'logreg__C': [8],
    'tf__num_bin__bi__n_bins': [10]}]

In [28]:
#Загрузка экземпляра класса перекрестной проверки
strat = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [29]:
#Модель GridSearchCV
gs = GridSearchCV(pipe, param_grid, scoring='roc_auc',cv=strat, return_train_score=False)

In [30]:
gs.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('tf',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('cat',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('imp',
                                                                                          SimpleImputer(add_indicator=False,
                                                                               

In [31]:
# Наилучшие значения гиперпараметров
print('Наилучшие значения гиперпараметров: {}'.format(gs.best_params_))
# Наилучшее значение правильности
print('Наилучшее значение правильности: {:.3f}'.format(gs.best_score_))
# Значение правильности
print('Значение правильности на тестовой выборке: {:.3f}'.format(gs.score(X_test, y_test)))

Наилучшие значения гиперпараметров: {'logreg__C': 8, 'tf__num_bin__bi__n_bins': 10}
Наилучшее значение правильности: 0.751
Значение правильности на тестовой выборке: 0.753
